In [1]:
from geopy.geocoders import Nominatim
from shapely.geometry import Point, LineString
import geopandas as gpd
import folium 
import matplotlib.pyplot as plt
import random
import pandas as pd

In [66]:
df = pd.read_csv('/Users/pauliuskomskis/Desktop/github/greenzone/data/2019/final_df.csv')

In [67]:
df.head().shape

(5, 14)

In [68]:
df.head()

,Unnamed: 0,city,population,violent_crime,murder,rape,robbery,assault,property_crime,burglary,larceny,gta,arson,Arson
0,0,Barre,8551,64,0,18,3,43,203,20,183,0,3.0,NaN
1,1,Barre Town,7679,6,0,1,0,5,67,3,62,2,1.0,NaN
2,2,Bellows Falls,2988,7,0,0,2,5,54,11,42,1,0.0,NaN
3,3,Bennington,14912,63,0,3,9,51,514,67,431,16,0.0,NaN
4,4,Berlin,2789,9,0,2,0,7,122,4,117,1,0.0,NaN


In [108]:
city_list = df['city']
city_list.shape

(7658,)

In [111]:
city_list.value_counts()

Springfield              14
Clinton                  14
Salem                    11
Newport                  11
Troy                     10
                         ..
Lumberton Township        1
Mountlake Terrace         1
Sleepy Hollow Village     1
Cottage City              1
Mazon                     1
Name: city, Length: 5992, dtype: int64

In [119]:
city_list.dtype

dtype('O')

In [121]:
def get_coordinates(city_list):
    """Takes a list of cities and returns a dictionary of the cities and their corresponding coordinates."""
    geolocator = Nominatim(user_agent='location script')
    dicto = {}

    for city in city_list:        
        try:

            location = geolocator.geocode(city)
        except Exception:
#             raise Exception("There was a problem with the getCoordinates function")
            pass
        coordinate_values = (location.longitude, location.latitude)  #in geopandas, the x value corresponds to the longitude while the y value, the latitude(Just in case you were wondering why it was *location.longitude, location.latitude* and not the other way round )
        dicto[city] = coordinate_values #adding the coordinate pair to the dictionary at the end of every loop
    return dicto #finally retruns the dict 

In [122]:
#getting coordinates for each city in the list
city_coords_dict = get_coordinates(city_list)
city_coords_dict

AttributeError: 'NoneType' object has no attribute 'longitude'

In [ ]:
len(city_coords_dict)

In [ ]:
df.head()

In [ ]:
# crime rate per 100,000 is made as follows:
#(crime volume / population) * 100,000


#### 1. Add a total crime column

In [ ]:
df['total_crime'] = df.sum(axis=1)

In [ ]:
df.head()

#### 2. Making a new data frame that has only the neccesary columns for further operations

In [ ]:
new_df = df.filter(['city','total_crime','population'], axis=1)
new_df

### 3.  Add a column of the crime rate severity via (total crime / population) 

In [ ]:
new_df['crime_index'] = (new_df['total_crime'] / new_df['population'])

In [ ]:
new_df.round(5)

#### Next step is to create a final dataframe with following columns:
#### LATITUDE, LONGTITUDE, CRIME_INDEX

In [ ]:
#lets make the coordinates dictionary into dataframe and orient it for
#latitudes and longitudes
coords_df = pd.DataFrame.from_dict(city_coords_dict, orient='index')
coords_df

In [ ]:
coords_df.rename(columns={0: 'latitude', 1: 'longitude'}, inplace=True)

In [ ]:
coords_df

### join original dataframe row of 'crime_index' to new df of 'coordinates'

In [ ]:
coords_df['crime_index'] = new_df['crime_index'].values

In [ ]:
coords_df